In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

In [62]:
%run functions
matplotlib.rcParams.update({'font.size': 25})
cm = pylab.get_cmap('tab10')
cm2 = pylab.get_cmap('Set1')
cm3 = pylab.get_cmap('turbo')
cm3j = pylab.get_cmap('jet')
cm4 = pylab.get_cmap('viridis')

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [11]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

# Use 20220617_extractdatatonpz before

# Data is in the file /home/adrien/RemoteFS/Rivendel/Simulations/20221017

time       = []
T_aux_avg  = []
r2_v2_rlim = []
alpha       = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])
        alpha.append(data['alpha'])

/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


In [12]:
myRF = 0

for k in range(3):
    print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]

R = r2_v2_rlim[myRF][6][50]*1e-3
L = r2_v2_rlim[myRF][8][50]*1e-3
aws = 0
print(R,L)
print((R+(1.48*aws/2))/(L+1.48*aws/2))

density = 1024 / (4/3*pi*R**2*L)

186.69759908981365
198.17204484455908
185.79916275551497
0.00018669759908981366 0.00018579916275551499
1.0048355241270968


In [13]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B
def dfunc5_dt(t,t0,A,B,C,k):
    return A*C*(abs(t-t0)**k+C)**( -(k+1)/k )
#     return A*(C+abs(t-t0)**k)**(-1/k)*(1 - ((t-t0)**2*abs(t-t0)**(k-2))/(C+abs(t-t0)**k) )

In [31]:
def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * exp(B)

In [15]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
1 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
2 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
3 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC05_RF05.npz
4 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


In [25]:
to_fit = 0
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
Urf = array([0, 0, 20.532292436697, 30.798438655045, 41.065, 51.33073109174249,0,0,6.1597])
q = array([0, 0, 0.2, 0.3, 0.4, 0.5, 0,0, 0.6])
Udc = array([0, 0, 1.3454211755646195, 3.062127421577093, 5.706, 8.837305210834145, 0,0, 13.08323245])
N_ions = array([1024,1024,1024,1024,1024])

beta_guess = 0.423
a = 0
for k in range(250):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)

omega_z_2 = ( 2*pi*100e3 )**2 * array(Udc)
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2
print(sqrt(omega_r_2[my_try])/2/pi)

x  = time[to_fit]-1e-3
fs = 1/diff(x)[1]

116592.34460485817


In [26]:
# Display R/L aspect ratio
for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    R = mean(r2_v2_rlim[cond][6][100:500])
    L = mean(r2_v2_rlim[cond][8][100:500])
    print(R/L)
    print(alpha[cond])
    print('')
address = file_path[0]
print(address)

1.0025291530048888
1.1091876490255044

1.0209015844409537
1.07676263747061

1.091920930542983
1.119602858673717

1.0817946388962292
1.107811977807591

1.1275838655493158
1.154081577392648

/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz


In [27]:
# Display parameters of simus
# in nice table

print('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|',end='\n')
print('|---------|----|----|---------|---------|---------|-----|-----|',end='\n')

for cond in range(len(file_path)):
    #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
    my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])
    
    print(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|',end='\n')

|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|
|---------|----|----|---------|---------|---------|-----|-----|
|DC02_RF02|1024|0.20|1.426e+05|1.160e+05|6.727e-03|1.109|0.143|
|DC03_RF03|1024|0.30|2.161e+05|1.750e+05|1.531e-02|1.077|0.216|
|DC04_RF04|1024|0.40|2.926e+05|2.389e+05|2.853e-02|1.120|0.293|
|DC05_RF05|1024|0.50|3.737e+05|2.973e+05|4.419e-02|1.108|0.374|
|DC08_RF08|1024|0.60|4.622e+05|3.617e+05|6.542e-02|1.154|0.462|


In [160]:
# write parameters in a file
with open('data_parameters.txt','w') as f:
    f.write('2022 oct 21\n')
    f.write('Paramètres des simulations de chauffage RF\n')
    f.write('Simulations dans le dossier Rivendel/Simulations/20221017\n\n')
    
    f.write('|   cond  | N  | q_x|   f_x   |   f_z   |   ~a_x  | R/L | beta|\n')
    f.write('|---------|----|----|---------|---------|---------|-----|-----|\n')

    for cond in range(len(file_path)):
        #print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]
        my_try = int(file_path[cond][file_path[cond].find('DC')+2:file_path[cond].find('DC')+4])

        f.write(f'|DC{my_try:02d}_RF{my_try:02d}|{N_ions[cond]:04d}|{q[my_try]:.2f}|{sqrt(omega_x_2)[my_try]/2/pi:.3e}|{sqrt(omega_z_2)[my_try]/2/pi:.3e}|{omega_z_2[my_try]/Omega**2*2:.3e}|{alpha[cond]:.3f}|{beta_guess[my_try]:.3f}|\n')

In [65]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 2 #11 19 8
temp_to_end = 50

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*2.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
# popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
#                               log10(y_filt[start_for_fit:end_fit]),
#                        p0, maxfev= 25000,
#                        bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
# print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

# y_fit5 = func5(x[start_for_fit:end_fit],*popt)
# ax1.semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)

ax1.set_xlabel('t [ms]')
ax1.set_ylabel('T [K]')
# ax1.set_xlim(-0.1,4.3)
ax1.set_ylim(2e-3,300)
ax1.legend()
tight_layout()


# savefig(figname+'.eps',dpi=300)

file to analyze
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz


In [32]:
start_for_fit = 10
T_crit = 0.5
print('file to analyze')
not_fitted = 0
# 29 and 30
to_fit = 0
temp_to_end = [100, 100, 100, 100, 100]
popt_smooth = np.zeros((len(file_path),5))
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
# retrive time and temperature from the curve
    print('>',to_fit,j)
    my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
    x  = time[to_fit]-1e-3
    y  = T_aux_avg[to_fit]

    # Smoothing signal with zero delay shift fit
    max_f_r = sqrt(omega_r_2[my_try])/2/pi
    b, a = signal.butter(8,  max_f_r/fs*1.01)
    y_filt = signal.filtfilt(b, a, y, padlen=150)
    
    # select index of temp where stop the fit
    # I stop the fit when T reaches a given T
    index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
    end_fit = index_100K

    # Fit parameters first guesses
    t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
    B0 = log(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
    A0 = log(max(y_filt[start_for_fit:end_fit]) - B0)

    # set initial guess of fit parameters
    p0 = [t0_first_guess,A0,B0,0.01,0.3]

    # proper fit of the smoothed curve limiter up to threshold T
    popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                  log(y_filt[start_for_fit:end_fit]),
                           p0, maxfev= 25000,
                           bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
    print(popt)
    popt_smooth[to_fit,:] = popt
# popt_smooth = popt

file to analyze
> 0 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
[2.32312736e-03 1.77473453e+01 7.55980280e-01 1.96284422e-01
 1.67227509e-01]
> 1 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
[1.23048096e-03 6.72932633e+00 4.57732273e-01 5.44620258e-02
 2.92564750e-01]
> 2 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
[0.08080208 4.48952946 0.61333532 0.0101186  0.43469374]
> 3 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC05_RF05.npz
[0.0858291  5.57872336 0.3978366  0.01956381 0.28214774]
> 4 /home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz
[6.72796518e-03 5.24940846e+00 9.03717848e-02 4.89443690e-03
 4.33048135e-01]


In [33]:
# when does fit crosses a given temperature
# compute delta T between this moment and t0
# computed above in the fit, from t=0
T_thre = 1e-2
fit_above_thresh = np.zeros((len(file_path)))
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
        x = time[to_fit]-1e-3
        y_fit = func5(x,*popt_smooth[to_fit,:])
        fit_above_thresh[to_fit] = x[where(min(abs(10**y_fit-T_thre))==abs(10**y_fit-T_thre))][0]
print(fit_above_thresh)

delta_T_explosion = popt_smooth[:,0] - fit_above_thresh

[0.00000000e+00 6.24500000e-07 8.02566245e-02 8.57366245e-02
 6.70412450e-03]


In [34]:
popt_rework = [popt_smooth[1:9],
              popt_smooth[12:20], # 7:15
              popt_smooth[20:]]

delta_T_explosion_rework = array([delta_T_explosion[1:9],
                            delta_T_explosion[12:20], # 7:15
                            delta_T_explosion[20:]])

#  3     4     5     6     7     8     9     10    11    12    13
# 31.0, 42.0, 55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8
#             0.54  0.56  0.58  0.6   0.62  0.64  0.66  0.68  0.7
Urf_rework = [[57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8],
             [57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8], # 57.4, 59.5, 
             [55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8,
                      64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47,
                      62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74]]
q_rework = [[0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7],
             [0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7 ], # 57.4, 59.5, 
             [0.54, 0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7,
                      0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.626, 0.628,
                      0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629]]
N_rework = [256, 512, 1024]


for k,l in enumerate(N_rework):
    print(f'fit_params_{l:04d}ions')
    np.savez( f'fit_params_{l:04d}ions', fit_param=popt_rework[k], Urf=Urf_rework[k], N=N_rework[k])

# func5(t, t0, A, B, C, k)
# with load('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/fit_params_0256ions.npz') as data:
#     a = data['N']
# print(a)

fit_params_0256ions
fit_params_0512ions
fit_params_1024ions


/tmp/ipykernel_26551/798389194.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  delta_T_explosion_rework = array([delta_T_explosion[1:9],


# From fluo-variations_optimal-temp.ipynb

# Functions used to describe ions
### From fluo-variations_optimal-temp.ipynb
I present the functions used to compute fluorescence and cooling power as a function of the temperature. See Foot chapter 7 and 9. The two basic functions are as follows.

- $\texttt{MB}(v)$ is the Maxwell-Boltzmann distribution for a given temperature. It provides the probability to find an atom with a given velocity in a gas with temperature T.

- $\texttt{pfl_dop}(v) = \rho_{ee}(v)$ is the atomic ray profile with Doppler effect. It provides the probability of excitation of a single oscillator given Rabi frequency, detuning, lambda and its velocity. It is considered equal to the excited population, i.e the proportion of excited atoms. This function is used with fixed Rabi frequency, detuning and lambda.

Those functions are used to compute the total fluorescence of the ion ensemble under laser cooling $F$, the cooling power of laser $G$, the heating induced by the spontaneous emission $G_{Hot}$.

- $\texttt{prob_fluo} = \texttt{pfl_dop}\times \texttt{MB}$ is the probability of excitation of an ion with velocity v in an ensemble at temperature $T$. The sum $F=\Gamma\sum_v \texttt{prob_fluo}\; dv$ is the fluorescence emitted by an ensemble of ions with given temperature $T$.

- $\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$ is used to compute the cooling energy of laser $G = \sum_v \texttt{cool_power}\; \mathrm{d}v \times \;\hbar k \Gamma$ [J].

- $\texttt{hot_power} = \texttt{prob_fluo}$ is used to compute the heating induced by the spontaneous emission $G_{Hot} = \sum_v \texttt{hot_power}\; \mathrm{d}v \times \;\hbar^2 k^2 \Gamma/m$ [J].

You can multiply $G$ and $G_{Hot}$ by $1/(k_b\tau_{RF}$ to express those quantities in K/RF period. This transformation relies on the following formula : E = 3/2 k_bT.


$\texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

$\texttt{cool_power} = \texttt{pfl_dop}\times \texttt{MB}\times kv$

In [35]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38064852*1e-23  # Boltzman
    m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def cool_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)*nu

def hot_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [36]:
T = 0.01
v_thermique = sqrt(2*kb*T/m_Ca)
nv = 2000
dv = 6*v_thermique/nv
dist_MB = []
for v in linspace(-3*v_thermique,3*v_thermique,nv):
    dist_MB.append(MB(v, T))
print(sum(dist_MB)*dv)

0.9994781288008786


In [37]:
# Laser parameters
lam = 397e-9 # m
klam = 2*pi/lam # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
Rab = Gam
I = 170 # W/m²

In [38]:
c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
Omega = 2.0e6*2*pi
tau_RF = 1/(Omega/(2*pi))

In [39]:
v_min = sqrt(h_pl_bar*Gam/m_Ca)
print(v_min)
print(m_Ca*v_min**2/kb)

0.4634206649198229
0.0010351970908243904


# Windowing

In [40]:
# Windowing temperature
window_samp = 200
x_win = [] # zeros((len(y)//window_samp,len(file_path)))
y_win = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path)):
#     print(to_fit)
    x_win.append([])
    y_win.append([])
    x = time[to_fit]-1e-3
    y = T_aux_avg[to_fit]
    for i in range(len(y)//window_samp-1):
        x_win[to_fit].append( (x[i*window_samp]+x[(i+1)*window_samp])/2 )
        y_win[to_fit].append( sum(y[i*window_samp:(i+1)*window_samp])/window_samp )
    
print(diff(x_win[0])[0]*1e3,tau_RF*window_samp*1e3)

0.10000000000000005 0.09999999999999999


In [41]:
# Plot windowed temperature
to_plot = 2
x  = time[to_plot]-1e-3
y  = T_aux_avg[to_plot]
figure('Windowed T',clear='True')
semilogy(x, y,':',color='y')
semilogy(x_win[to_plot], y_win[to_plot],'+')
# hlines(500,0,0.02)
grid()
tight_layout()

In [74]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 500
log_temps = linspace(-3.1,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
coolpow_4 = []
coolpow_6 = []
# coolpow_8 = []
coolpow_10 = []
# coolpow_50 = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 6*v_thermique/nv
    coolpow_05.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

my_color=[7,2,6,3,3] # 
figname = 'Windowed_dT_dt_q_N'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [0,1,2,3,4]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=14,mec='k',mew=0.2,ls='',color=cm4((incr+0.3)/4),
              label=f'{q[my_try]}')
    incr+=1

max_H = 50
ax1.loglog(y_win[0][max_H-50:max_H+15-1], diff(y_win[0][max_H-50:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2,color=cm4((0+0.3)/4))
max_H = 10
ax1.loglog(y_win[1][max_H-10:max_H+15-1], diff(y_win[1][max_H-10:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2,color=cm4((1+0.3)/4))  
max_H = 815
ax1.loglog(y_win[2][max_H-50:max_H+15-1], diff(y_win[2][max_H-50:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2,color=cm4((2+0.3)/4))
max_H = 850
ax1.loglog(y_win[3][max_H-15:max_H+15-1], diff(y_win[3][max_H-15:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2,color=cm4((3+0.3)/4))
max_H = 65
ax1.loglog(y_win[4][max_H-15:max_H+15-1], diff(y_win[4][max_H-15:max_H+15])/window_samp,
      marker=' ',ls=':',lw=2,color=cm4((4+0.3)/4))  
# loglog(0,0,' ',label=' ')
# to_plot = 20
# ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#           marker='.',ms=14,mec='k',mew=0,ls='',color=cm(4/8),
#           label=f'{N_rework[2]}')

# ax1.loglog(y_win[choosen_plot[0]][:-1],
#            diff(y_win[choosen_plot[0]][:])/window_samp,
#           marker='',ls=':',color=cm(0/8))
    
ax1.loglog(10**log_temps,coolpow_05,ls="-",color='k',lw=2,label='$-0.5\Gamma$')
ax1.loglog(10**log_temps,coolpow_1,ls="--",color='k',lw=2,label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_1_bis,ls="--",color='r',label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_2,ls=":",color='xkcd:rouge',lw=2.5,label='$-2\Gamma$')
ax1.loglog(10**log_temps,coolpow_4,ls=":",color=cm3j(3/10),lw=5,label='$-4\Gamma$')  
# ax1.loglog(10**log_temps,coolpow_6,ls=":",color='xkcd:green',lw=2.5,label='$-6\Gamma$')
# ax1.loglog(10**log_temps,coolpow_8,ls=":",color='xkcd:blue',label='$-8\Gamma$')  
ax1.loglog(10**log_temps,coolpow_10,ls=":",color=cm3j(9/10),lw=5,label='$-10\Gamma$')
# ax1.loglog(10**log_temps,coolpow_50,ls=":",color='xkcd:black',label='$-50\Gamma$') 
    
ax1.set_xlabel('T (K)')
ax1.set_ylabel('$\mathrm{d}T/\mathrm{d}t$ (K/RF period)')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=f'$q_x$, $N=1024$',loc=4,ncol=2,fontsize=18)
ax1.set_xlim(7e-3,0.6e3)
ax1.set_ylim(1e-9,6e-1)

tight_layout()

savefig(figname+'qvar'+'.eps',dpi=300)
savefig(figname+'qvar'+'.jpg',dpi=300)

/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC05_RF05.npz
/home/adrien/RemoteFS/Rivendel/Simulations/20221017/Time_and_temp_RFHEAT_N1024_DC08_RF08.npz


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
